In [ ]:
# !pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!gdown 1izxctyA4b0anBkDgTrpppirwEXgSAwEY

filepath = '/content/web_data.csv'
df = pd.read_csv(filepath)
df.drop_duplicates(keep='first', inplace=True)
df.drop("Location", axis = 1, inplace = True)
print(df.shape)
df.head()

Downloading...
From: https://drive.google.com/uc?id=1izxctyA4b0anBkDgTrpppirwEXgSAwEY
To: /content/web_data.csv
100% 2.36M/2.36M [00:00<00:00, 197MB/s]
(10229, 2)


,Category,Incident
0,Indecent exposure,I'm a 58 year old woman and not used to seein...
1,Touching /Groping,a boy with whom I was in relationship with ca...
2,Stalking,I was stalked in this street and cat-called. ...
3,Touching /Groping,groped in the middle of the road \n\t\t\t\t\...
4,Stalking,I go to the race course for early morning run...


In [ ]:
df['Category'] = df['Category'].astype("category")

In [ ]:
num_of_labels = len(df['Category'].unique())

In [ ]:
df['Incident'][0]

" I'm a 58 year old woman and not used to seeing such happenings, even though I have had my share of being groped, touched inappropriately, felt...  \n\t\t\t\t\t\t  More Information »\n« Less Information\n"

In [ ]:
unwanted = "!@#$;:!*%)(&^~,."

for i in range(len(df['Incident'])):
    try:
        s = df['Incident'][i]
        s = s.split('\n')[0]
        s = s.strip()
        for char in unwanted:
            s = s.replace(char,"")
        df.loc[i, 'Incident'] = s.lower()
    except:
        pass

df.drop(df.index[(df["Incident"] == "na")],axis=0,inplace=True)
df.head()

,Category,Incident
0,Indecent exposure,i'm a 58 year old woman and not used to seeing...
1,Touching /Groping,a boy with whom i was in relationship with cal...
2,Stalking,i was stalked in this street and cat-called
3,Touching /Groping,groped in the middle of the road
4,Stalking,i go to the race course for early morning runs...


In [ ]:
X = df['Incident']
y = df['Category']

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [ ]:
X_train = X_train.tolist()
y_train = y_train.tolist()
X_test = X_test.tolist()
y_test = y_test.tolist()

In [ ]:
# from transformers import DistilBertTokenizerFast

# tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
# inputs = tokenizer(X_train, padding="max_length", truncation=True)

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
inputs = tokenizer(X_train, padding="max_length", truncation=True)

In [ ]:
import torch

class feedbackDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = feedbackDataset(inputs, y_train)

In [ ]:
# print(train_dataset.__getitem__(2))

In [ ]:
# from transformers import DistilBertForSequenceClassification

# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = num_of_labels)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [ ]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = num_of_labels)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
            output_dir = './results', 
            num_train_epochs = 6,
            per_device_train_batch_size = 16,
            per_device_eval_batch_size = 64, 
            warmup_steps = 500,
            weight_decay = 0.01,
            logging_dir = './logs', 
            logging_steps = 10
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
)

trainer.train()

Step,Training Loss
10,2.336100
20,2.308800
30,2.242400
40,2.160200
50,2.112200
60,2.190600
70,2.047500
80,2.132100
90,2.009200
100,2.154500


TrainOutput(global_step=2862, training_loss=1.086622454751879, metrics={'train_runtime': 4132.777, 'train_samples_per_second': 11.076, 'train_steps_per_second': 0.693, 'total_flos': 1.204472679837696e+16, 'train_loss': 1.086622454751879, 'epoch': 6.0})

In [ ]:
def to_check_result(test_encoding):

    input_ids = torch.tensor(test_encoding['input_ids']).to(device)
    attention_mask = torch.tensor(test_encoding['attention_mask']).to(device)

    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
    
    y = np.argmax(outputs[0].to('cpu').numpy())

    return y

In [ ]:
l2 = []

for i in X_test:

    test_encoding1 = tokenizer(i, truncation=True, padding=True)
    op = to_check_result(test_encoding1)
    l2.append(op)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, l2)
acc

0.5742924528301887

**DistilBERT** - 0.5762 <br>
**RoBERTa** - 0.5824<br>
**DeBERTa** - CUDA out of memory<br>